In [1]:
import sys
!{sys.executable} -m pip install csv_to_sqlite

You are using pip version 9.0.1, however version 20.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import sqlite3
conn = sqlite3.connect('air.db')
c = conn.cursor()
c.execute("select * from Asthma")

In [3]:
import csv_to_sqlite
from pathlib import Path
Path('air.db').touch()
import pandas as pd
asthma = pd.read_csv("Asthma & Air Quality Data.csv")
asthma.to_sql('ast', conn, if_exists='append', index = False)

c.execute('''SELECT * FROM ast''').fetchall()

/Users/TinaShahid/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


[('Bronx',
  'Crotona - Tremont',
  40.8462289,
  -73.890437,
  'Fine Particulate Matter (PM2.5)',
  8.2,
  '0 to 17 years',
  '3,161'),
 ('Bronx',
  'Fordham - Bronx Pk',
  40.86662962,
  -73.88303046,
  'Fine Particulate Matter (PM2.5)',
  7.6,
  '0 to 17 years',
  '3,120'),
 ('Bronx',
  'High Bridge - Morrisania',
  40.844632,
  -73.922661,
  'Fine Particulate Matter (PM2.5)',
  8.2,
  '0 to 17 years',
  '3,559'),
 ('Bronx',
  'Hunts Point - Mott Haven',
  40.82028067,
  -73.88318712,
  'Fine Particulate Matter (PM2.5)',
  8.3,
  '0 to 17 years',
  '2,486'),
 ('Bronx',
  'Kingsbridge - Riverdale',
  40.88661177,
  -73.89968457,
  'Fine Particulate Matter (PM2.5)',
  7.5,
  '0 to 17 years',
  '336'),
 ('Bronx',
  'Northeast Bronx',
  40.88961947,
  -73.82267709,
  'Fine Particulate Matter (PM2.5)',
  7.5,
  '0 to 17 years',
  '1,467'),
 ('Bronx',
  'Pelham - Throgs Neck',
  40.81128266,
  -73.82623649,
  'Fine Particulate Matter (PM2.5)',
  7.5,
  '0 to 17 years',
  '2,356'),
 ('Broo

In [4]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster

asthma = pd.read_csv("Asthma & Air Quality Data.csv")
asthma.head()

,Borough,Geography - UHF 42,Latitude,Longitude,Air Quality Indicator,"Mean, mcg per cubic meter",Age Group,Asthma ER visits
0,Bronx,Crotona - Tremont,40.846229,-73.890437,Fine Particulate Matter (PM2.5),8.2,0 to 17 years,"3,161"
1,Bronx,Fordham - Bronx Pk,40.866630,-73.883030,Fine Particulate Matter (PM2.5),7.6,0 to 17 years,"3,120"
2,Bronx,High Bridge - Morrisania,40.844632,-73.922661,Fine Particulate Matter (PM2.5),8.2,0 to 17 years,"3,559"
3,Bronx,Hunts Point - Mott Haven,40.820281,-73.883187,Fine Particulate Matter (PM2.5),8.3,0 to 17 years,"2,486"
4,Bronx,Kingsbridge - Riverdale,40.886612,-73.899685,Fine Particulate Matter (PM2.5),7.5,0 to 17 years,336


In [5]:

map = folium.Map(location=[40.7128, -74.0060], zoom_start = 10)

# Display the map
map.save('mymap.html')
map

In [6]:
map.choropleth(geo_data="Asthma.geojson",
             #data= asthma, # my dataset SQL lite dataset here
             #df['Asthma ER visits']=df['Asthma ER visits'].astype(str),
             columns=['Geography - UHF 42', 'Asthma ER visits'], # geography is here for matching the geojson geography, er visits is the column that changes the color of er visits areas
             key_on='feature.properties.Borough', # this path contains zipcodes in str type, this zipcodes should match with our ZIP CODE column
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2)
             #legend_name='Asthma ER Visits')

map.save('mymap.html')
map

In [7]:
marker_cluster = MarkerCluster().add_to(map) # create marker clusters


In [8]:
for i in range(asthma.shape[0]):
    location = [asthma['Latitude'][i],asthma['Longitude'][i]]
    tooltip = "Borough:{}<br> Borough: {}<br> Click for more".format(asthma["Borough"][i], asthma['Asthma ER visits'][i])    
    folium.Marker(map, # adding more details to the popup screen using HTML
                  popup="""
                  <i>Borough: </i> <br> <b>${}</b> <br> 
                  <i>Asthma ER Visits: </i><b><br>{}</b><br>
                  <i>mean square feet: </i><b><br>{}</b><br>""".format,
                 
                  tooltip=tooltip).add_to(marker_cluster)

TypeError: 'Map' object is not iterable

In [ ]:
map 